### Data Transformation

In [1]:
import os
%pwd

'd:\\MLOPS\\TextSummrizer\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\MLOPS\\TextSummrizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )    
        return data_transformation_config

In [9]:
import os
from src.TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

d:\MLOPS\TextSummrizer\summrizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Transformation Component

In [14]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

        target_encodings=self.tokenizer(text_target=example_batch['summary'],max_length=128,truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }  

    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path) 
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)  
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))    

In [15]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()

[2026-02-12 02:01:27,345: INFO: common : yaml file: config\config.yaml loaded successfully]
[2026-02-12 02:01:27,352: INFO: common : yaml file: params.yaml loaded successfully]
[2026-02-12 02:01:27,353: INFO: common : created directory at: artifacts]
[2026-02-12 02:01:27,357: INFO: common : created directory at: artifacts/data_transformation]


[2026-02-12 02:01:27,792: INFO: _client : HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-12 02:01:27,824: INFO: _client : HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json "HTTP/1.1 200 OK"]
[2026-02-12 02:01:28,126: INFO: _client : HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-12 02:01:28,189: INFO: _client : HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/tokenizer_config.json "HTTP/1.1 200 OK"]
[2026-02-12 02:01:28,480: INFO: _client : HTTP Request: GET https://huggingface.co/api/models/google/pegasus-cnn_dailymail/tree/main/additional_chat_templates?recursive=false&expand=false "HTTP/1.1 404 Not Found"]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 54000.80 examples/s] 
